# Exploring VB library

In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
from copy import deepcopy

import VI_algorithms_lib
import valez_finite_VI_lib as vi

import sys
sys.path.append('../../LinearResponseVariationalBayes.py')

from VariationalBayes.ParameterDictionary import ModelParamsDict
from VariationalBayes.DirichletParams import DirichletParamArray
from VariationalBayes.NormalParams import MVNArray
from VariationalBayes.Parameters import ArrayParam, ScalarParam

import generic_optimization_lib as packing

# Draw data

In [2]:
np.random.seed(4368280)


Num_samples = 10 # sample size
D = 2 # dimension
# so X will be a N\times D matrix

K_inf = 3 # take to be large for a good approximation to the IBP

alpha = 10.0 # IBP parameter

# Parameters to draw A from MVN
sigma_eps = 0.1 # variance of noise
sigma_A = 100.0

# generate data
Pi_truth, Z_truth, A_truth, X = vi.generate_data(Num_samples, D, K_inf, sigma_A, sigma_eps, alpha)

K_approx = deepcopy(K_inf) # variational truncation
print(Pi_truth)

[ 0.64604807  0.96402149  0.89696843]


In [3]:
## define variational parameters
vb_params = ModelParamsDict(name = 'vb_params')
# stick lengths
vb_params.push_param(DirichletParamArray(name='pi', shape=(K_approx, 2))) 
# variational means
vb_params.push_param(MVNArray(name='phi', shape=(K_approx, D))) 
# responsibilities
vb_params.push_param(ArrayParam(name = 'nu', shape = (Num_samples, K_approx), lb = 0.0, ub = 1.0)) 

# initialize
tau_init, nu_init, phi_mu_init, phi_var_init = vi.initialize_parameters(Num_samples, D, K_approx)
    
vb_params['phi'].set_vector(np.hstack([np.ravel(phi_mu_init.T), phi_var_init]))
vb_params['pi'].set_vector(np.ravel(tau_init))
vb_params['nu'].set_vector(np.ravel(nu_init))

In [4]:
# double check set_vector and np.ravel is doing what I expect
print('check nu \n')
print(nu_init)
print(vb_params['nu'].get(), '\n')

print('check beta means \n')
#print(tau_init / np.sum(tau_init,1)[:,None])
#print(vb_params['pi'].e(), '\n')
print(tau_init)
print(vb_params['pi'].alpha.get())

print('check normal means \n')
print(phi_mu_init.T)
print(vb_params['phi'].e(), '\n')

print('check that this matches with pack_params \n')
print(vb_params.get_free())
print(packing.pack_params(tau_init, phi_mu_init, phi_var_init, nu_init))

check nu 

[[ 0.75850642  0.08465173  0.74905987]
 [ 0.17853003  0.41449694  0.1614054 ]
 [ 0.28504637  0.57223453  0.15204038]
 [ 0.05121825  0.52189681  0.07056183]
 [ 0.71037797  0.45740886  0.84457221]
 [ 0.54342127  0.42040212  0.34355862]
 [ 0.31107763  0.23813339  0.45629593]
 [ 0.07362009  0.82848963  0.36932312]
 [ 0.38967501  0.534157    0.01615141]
 [ 0.45087606  0.61555167  0.31620724]]
[[ 0.75850642  0.08465173  0.74905987]
 [ 0.17853003  0.41449694  0.1614054 ]
 [ 0.28504637  0.57223453  0.15204038]
 [ 0.05121825  0.52189681  0.07056183]
 [ 0.71037797  0.45740886  0.84457221]
 [ 0.54342127  0.42040212  0.34355862]
 [ 0.31107763  0.23813339  0.45629593]
 [ 0.07362009  0.82848963  0.36932312]
 [ 0.38967501  0.534157    0.01615141]
 [ 0.45087606  0.61555167  0.31620724]] 

check beta means 

[[ 1.20509906  0.80452904]
 [ 0.60422205  1.62573816]
 [ 1.97115228  1.63332368]]
[[ 1.20509906  0.80452904]
 [ 0.60422205  1.62573816]
 [ 1.97115228  1.63332368]]
check normal means 

[

In [5]:
print(alpha)

10.0


In [6]:
# define hyper parameters
hyper_params = ModelParamsDict('hyper_params')
hyper_params.push_param(ScalarParam('alpha', lb = 0.0))
hyper_params.push_param(ScalarParam('var_a', lb = 0.0))
hyper_params.push_param(ScalarParam('var_eps', lb = 0.0))


hyper_params['alpha'].set(alpha)
hyper_params['var_a'].set(sigma_A)
hyper_params['var_eps'].set(sigma_eps)



# just checking ...
print(packing.pack_hyperparameters(alpha, sigma_A, sigma_eps))
print(hyper_params.get_vector())

[  10.   100.     0.1]
[  10.   100.     0.1]


In [7]:
# check the elbo computation
model = vi.IBP_Finite_Model(X, vb_params, hyper_params)
model.elbo()

[-0.71157178 -2.08667144 -0.73308428]
[-1.38361452 -0.4134043  -0.98282395]
[[-0.67875587 -0.12046844  0.25910884]
 [-1.01534522  2.17037139 -0.2226591 ]]
[[ 1.46070953  1.01451264  1.06713739]
 [ 2.03092592  5.71051195  1.04957707]]
[[ 0.75850642  0.08465173  0.74905987]
 [ 0.17853003  0.41449694  0.1614054 ]
 [ 0.28504637  0.57223453  0.15204038]
 [ 0.05121825  0.52189681  0.07056183]
 [ 0.71037797  0.45740886  0.84457221]
 [ 0.54342127  0.42040212  0.34355862]
 [ 0.31107763  0.23813339  0.45629593]
 [ 0.07362009  0.82848963  0.36932312]
 [ 0.38967501  0.534157    0.01615141]
 [ 0.45087606  0.61555167  0.31620724]]
100.0
0.1
10.0
1462.01216815
24.1911468518


1486.2033150038146

In [8]:
sigmas = {'eps': sigma_eps, 'A': sigma_A}
vi.compute_elbo(tau_init, nu_init, phi_mu_init, phi_var_init, X, sigmas, alpha)

[-0.71157178 -2.08667144 -0.73308428]
[-1.38361452 -0.4134043  -0.98282395]
[[-0.67875587 -0.12046844  0.25910884]
 [-1.01534522  2.17037139 -0.2226591 ]]
[[ 1.46070953  1.01451264  1.06713739]
 [ 2.03092592  5.71051195  1.04957707]]
[[ 0.75850642  0.08465173  0.74905987]
 [ 0.17853003  0.41449694  0.1614054 ]
 [ 0.28504637  0.57223453  0.15204038]
 [ 0.05121825  0.52189681  0.07056183]
 [ 0.71037797  0.45740886  0.84457221]
 [ 0.54342127  0.42040212  0.34355862]
 [ 0.31107763  0.23813339  0.45629593]
 [ 0.07362009  0.82848963  0.36932312]
 [ 0.38967501  0.534157    0.01615141]
 [ 0.45087606  0.61555167  0.31620724]]
100.0
0.1
10.0
1462.01216815
24.1911468518


1486.2033150038146